# **파이썬으로 email 보내기**
업무 자동화는 많은 사람들이 파이썬을 배우는 이유이기도 하다. 특히 python으로 email을 보내는 방법을 알아두면 유용하게 사용할 수 있다. 광고 메일을 보내는 것은 물론이고, 나 자신이나 다른 사람들에게 자동화 메일을 보낼 수도 있다. 

예를 들어 주기적으로 하는 특정 작업을 실행하고 결과를 이메일로 보낼 수도 있고, 머신러닝 학습이 끝나면 결과 데이터를 자동으로 파일 첨부하여 자신의 메일로 받아볼 수도 있을 것이다.

## 1. Preparation

### 1.1 Importing libraries

In [1]:
import os, pickle
import smtplib                                   # smtplib: 메일 전송을 위한 패키지
from email import encoders                       # 파일전송을 할 때 이미지나 문서 동영상 등의 파일을 문자열로 변환할 때 사용할 패키지
from email.mime.text import MIMEText             # 본문내용을 전송할 때 사용되는 모듈
from email.mime.multipart import MIMEMultipart   # 복잡한 메시지 전송 시 사용하는 모듈
from email.mime.base import MIMEBase             # 파일을 전송할 때 사용되는 모듈

### 1.2 Setting login information: email & password
- password는 직접 노출시 유출의 위험이 있으므로 아래와 같이 pickle 파일로 저장해 이용하는 것이 좋다.
- 이 때, 본인의 지메일 비밀번호로는 보안상의 이유로 접근이 제한될 수 있다. 그럴 땐 앱 비밀번호를 발급([참고](https://pdi-mz-support.zendesk.com/hc/ko/articles/360050293051--Gmail-2%EB%8B%A8%EA%B3%84-%EC%9D%B8%EC%A6%9D-%EC%82%AC%EC%9A%A9-%EC%8B%9C-%EC%95%B1-%EB%B9%84%EB%B0%80%EB%B2%88%ED%98%B8-%EC%83%9D%EC%84%B1%ED%95%98%EC%97%AC-%ED%83%80-%ED%81%B4%EB%9D%BC%EC%9D%B4%EC%96%B8%ED%8A%B8-%EB%B3%B4%EC%95%88%EC%88%98%EC%A4%80%EC%9D%B4-%EB%82%AE%EC%9D%80-%EC%95%B1-%EC%97%B0%EA%B2%B0%ED%95%98%EA%B8%B0))해서 이용해보자.

In [2]:
# pw = "[your_password]"
# pickle.dump(pw, open("./pw.pickle", 'wb'))

email = "hyeshinoh@gmail.com"
pw = pickle.load(open('./pw.pickle', 'rb'))

In [3]:
# 메일을 보낼 주소
toAddr = ["panda706@naver.com", "hyeshinoh+11@gmail.com", "hyeshinoh+h@gmail.com"]

## 2. Send text

### 2.1 Access smtp server

In [4]:
# Access smtp server
# smtp obj. 생성
smtp = smtplib.SMTP('smtp.gmail.com', 587)   # 587: 서버의 포트번호
smtp.ehlo()
smtp.starttls()          # tls방식으로 접속, 그 포트번호가 587
smtp.login(email, pw)    # 사용자 인증

(235, b'2.7.0 Accepted')

### 2.2 Make message

msg object를 생성하고 제목을 넣어준다.

In [5]:
# Make Msg
msg = MIMEMultipart()    #msg obj.
msg['Subject'] = 'SMTP Send Text 테스트'

msg object에 본문(text msg)을 추가해준다.

In [6]:
# text msg
part = MIMEText('SMTP로 HTML 메일 보내기 본문 메시지입니다.')
msg.attach(part)   # msg에 part obj.를 추가해줌
msg

### 2.3 email 전송하기

In [7]:
# email 전송
for addr in toAddr:
    msg["To"] = addr
    smtp.sendmail("hyeshinoh@gmail.com", addr, msg.as_string())   # object인 msg를 .as_string()으로 문자열 parsing
    print(addr)

panda706@naver.com
hyeshinoh+11@gmail.com
hyeshinoh+h@gmail.com


### 2.4 HTML code 추가하기

In [13]:
# html 코드 추가
part_html = MIMEText('<br><a href="https://github.com/hyeshinoh/">hyeshin github</a>', 'html')
msg.attach(part_html)
msg  #msg 상태: 제목, 본문, html 코드

### 2.5 email 전송하기

In [12]:
# email 전송
for addr in toAddr:
    msg["To"] = addr
    smtp.sendmail(email, addr, msg.as_string())
    print(addr)

panda706@naver.com
hyeshinoh+11@gmail.com
hyeshinoh+h@gmail.com


In [14]:
# smtp close
smtp.quit()

(221, b'2.0.0 closing connection z90-v6sm19336456pfk.85 - gsmtp')

## 3. Send File

### 3.1 Access smtp server

In [15]:
# Access smtp server
smtp = smtplib.SMTP('smtp.gmail.com', 587)   # 587: 서버의 포트번호
smtp.ehlo()
smtp.starttls()   
smtp.login(email, pw)

(235, b'2.7.0 Accepted')

### 3.2 Make message

msg object를 생성하고 제목을 넣어준다.

In [16]:
# Make Msg
msg = MIMEMultipart()
msg['Subject'] = 'SMTP send file 테스트(제목)'

In [18]:
# text msg
part = MIMEText('SMTP로 send file 본문 메시지')
msg.attach(part)

In [19]:
# Attach FILE  - javascript file은 보안 상 이유로 보낼 수 없게 되어 있음
path = 'video.mp4'
# ctype = 'application/octet-stream'
# maintype, subtype = ctype.split('/', 1)
with open(path, 'rb') as f:
    part = MIMEBase("application", "octet-stream")
    part.set_payload(f.read())    #payload: osi 7-layers
    encoders.encode_base64(part)  #base64 encoding: 영상, 이미지 파일을 문자열 형태로 변환
    part.add_header('Content-Disposition', 'attachment', filename=path)
    msg.attach(part)
msg  # 제목, 본문, 파일

### 3.3 email 전송하기

In [20]:
for addr in toAddr:
    msg["To"] = addr
    smtp.sendmail(email, addr, msg.as_string())
    print(addr)

panda706@naver.com
hyeshinoh+11@gmail.com
hyeshinoh+h@gmail.com


In [21]:
# smtp close
smtp.quit()

(221, b'2.0.0 closing connection d22-v6sm9509327pfk.69 - gsmtp')